# Use Model Context Protocol (MCP) as tools with Strands Agent

## Overview
The [Model Context Protocol (MCP)](https://modelcontextprotocol.io/introduction) is an open protocol that standardizes how applications provide context to Large Language Models (LLMs). Strands AI SDK integrates with MCP to extend agent capabilities through external tools and services.

MCP enables communication between agents and MCP servers that provide additional tools. The Strands Agent SDK includes built-in support for connecting to MCP servers and using their tools.

In this example we will show you how to use MCP tools on your Strands Agent. We will use the [AWS Documentation MCP server](https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server/) which provides tools to access AWS documentation, search for content, and get recommendations. This MCP server has 3 main features:

- **Read Documentation**: Fetch and convert AWS documentation pages to markdown format
- **Search Documentation**: Search AWS documentation using the official search API
- **Recommendations**: Get content recommendations for AWS documentation pages



## Agent Details
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Feature used        |MCP Tools                                          |
|Agent Structure     |Single agent architecture                          |

</div>

## Architecture

<div style="text-align:center">
    <img src="./images/architecture.png" width="65%" />
</div>

## Key Features
* **Single agent architecture**: this example creates a single agent that interacts with MCP tools
* **MCP tools**: Integration of MCP tools with your agent

## Setup and prerequisites

### Prerequisites
* Python 3.10+
* AWS account
* Anthropic Claude 3.7 enabled on Amazon Bedrock

Let's now install the requirement packages for our Strands Agent agent

In [2]:
# installing pre-requisites
# !pip install -r requirements.txt

### Importing dependency packages

Now let's import the dependency packages

In [1]:
import threading
import time
from datetime import timedelta

from mcp import StdioServerParameters, stdio_client
from mcp.client.streamable_http import streamablehttp_client
from mcp.server import FastMCP
from strands import Agent
from strands.tools.mcp import MCPClient
from strands.models import BedrockModel
model_id = "anthropic.claude-3-5-sonnet-20241022-v2:0"
model = BedrockModel(
    model_id=model_id,
)

### Connect to MCP server using stdio transport

[Transports](https://modelcontextprotocol.io/specification/2025-03-26/basic/transports) in MCP provide the foundations for communication between clients and servers. It handles the underlying mechanics of how messages are sent and received. At the moment there are three standards transport implementations built-in in MCP:

- **Standard Input/Output (stdio)**: enables communication through standard input and output streams. It is particularly useful for local integrations and command-line tools
- **Streamable HTTP**: this replaces the HTTP+SSE transport from previous protocol version. In the Streamable HTTP transport, the server operates as an independent process that can handle multiple client connections. This transport uses HTTP POST and GET requests. Server can optionally make use of Server-Sent Events (SSE) to stream multiple server messages. This permits basic MCP servers, as well as more feature-rich servers supporting streaming and server-to-client notifications and requests.
- **SSE**: legacy transport for HTTP-based MCP servers that use Server-Sent Events transport  

Overall, you should use stdio for building command-line tools, implementing local integrations and working with shell scripts. You should use Streamable HTTP transports when you need a flexible and efficient way for AI agents to communicate with tools and services, especially when dealing with stateless communication or when minimizing resource usage is crucial.

You can also use **custom transports** implementation for your specific needs. 


Let's now connect to the MCP server using stdio transport. First of all, we will use the class `MCPClient` to connect to the [AWS Documentation MCP Server](https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server/). This server provides tools to access AWS documentation, search for content, and get recommendations.

In [2]:
# Connect to an MCP server using stdio transport
stdio_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

#### Setup agent configuration and invoke it

Next we will set our agent configuration using the tools from the `stdio_mcp_client` object we just created. To do so, we need to list the tools available in the MCP server. We can use the `list_tools_sync` method for it. 

After that, we will ask a question to our agent.

In [3]:
# Create an agent with MCP tools
with stdio_mcp_client:
    # Get the tools from the MCP server
    tools = stdio_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(
        model= model,
        tools=tools)

    response = agent("What is Amazon Bedrock pricing model. Be concise.")

Let me search for AWS Bedrock pricing documentation.
Tool #1: search_documentation
Let me get the specific pricing details.
Tool #2: read_documentation
Based on the documentation, here's the concise explanation of Amazon Bedrock's pricing model:

1. Pay-as-you-go based on usage:
   - Charged for input tokens
   - Charged for output tokens
2. Optional Provisioned Throughput purchase available for increased model invocation capacity
3. Pricing varies by specific model being used
4. You only pay for what you use, with no upfront commitments required

Each foundation model has its own pricing rates, which can be found in the Model providers page in the Amazon Bedrock console.

### Connect to MCP server using Streamable HTTP

Let's now connect to the MCP server using Streamable HTTP transport. First let's start a simple MCP server using Streamable HTTP transport. 

For this example we will create our own MCP server. The architecture will look as following

<div style="text-align:center">
    <img src="./images/architecture_2.png" width="65%" />
</div>

In [4]:
# Create an MCP server
mcp = FastMCP("Calculator Server")

# Define a tool


@mcp.tool(description="Calculator tool which performs calculations")
def calculator(x: int, y: int) -> int:
    return x + y


@mcp.tool(description="This is a long running tool")
def long_running_tool(name: str) -> str:
    time.sleep(25)
    return f"Hello {name}"


def main():
    mcp.run(transport="streamable-http", mount_path="mcp")

Let's now start a thread with the `streamable-http` server

In [5]:
thread = threading.Thread(target=main)
thread.start()

INFO:     Started server process [182505]
INFO:     Waiting for application startup.


[10/29/25 06:19:25] INFO     StreamableHTTP session manager started                  ]8;id=464839;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=651994;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/server/streamable_http_manager.py#110\110]8;;\

INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:49928 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:49936 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:49940 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:49952 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:49958 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:57320 - "DELETE /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:47266 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:47270 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:47282 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:47296 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:47306 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:47322 - "DELETE /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:38188 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:38192 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:38196 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:38202 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:45650 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:45654 - "

#### Integrating Streamable HTTP client with Agent

Now let's use `streamablehttp_client` integrate this server with a simple agent. 

In [6]:
def create_streamable_http_transport():
    return streamablehttp_client("http://localhost:8000/mcp")


streamable_http_mcp_client = MCPClient(create_streamable_http_transport)

#### Setup agent configuration and invoke it

Next we will set our agent configuration using the tools from the `streamable_http_mcp_client` object we just created. To do so, we need to list the tools available in the MCP server. We can use the `list_tools_sync` method for it. 

After that, we will ask a question to our agent.

In [7]:
with streamable_http_mcp_client:
    tools = streamable_http_mcp_client.list_tools_sync()

    agent = Agent(
        model=model,
        tools=tools)

    response = str(agent("What is 2 + 2?"))

[10/29/25 06:20:22] INFO     Created new transport with session ID:                  ]8;id=49154;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=852723;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/server/streamable_http_manager.py#233\233]8;;\
                             6c0a94598e5d44a6a028f1ac3d8056d4                                                      

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=964493;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=602330;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Received session ID: 6c0a94598e5d44a6a028f1ac3d8056d4           ]8;id=866070;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=509646;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/client/streamable_http.py#134\134]8;;\

                    INFO     Negotiated protocol version: 2025-06-18                         ]8;id=112504;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=860326;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/client/streamable_http.py#146\146]8;;\

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 202 Accepted"   ]8;id=90267;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=68379;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/httpx/_client.py#1740\1740]8;;\

...

[10/29/25 06:20:23] INFO     Processing request of type CallToolRequest                               ]8;id=682015;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=216578;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/server/lowlevel/server.py#664\664]8;;\

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=213682;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=170223;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/httpx/_client.py#1740\1740]8;;\

2 + 2 = 4

[10/29/25 06:20:24] INFO     Terminating session: 6c0a94598e5d44a6a028f1ac3d8056d4           ]8;id=784334;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/server/streamable_http.py\streamable_http.py]8;;\:]8;id=314541;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/server/streamable_http.py#630\630]8;;\

                    INFO     HTTP Request: DELETE http://localhost:8000/mcp "HTTP/1.1 200 OK"       ]8;id=488888;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=419465;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/httpx/_client.py#1740\1740]8;;\

### Direct Tool Invocation

While tools are typically invoked by the agent based on user requests, you can also call MCP tools directly. This can be useful for workflow scenarios where you orchestrate multiple tools together.

In [8]:
query = {"x": 10, "y": 20}

with streamable_http_mcp_client:
    # direct tool invocation
    result = streamable_http_mcp_client.call_tool_sync(
        tool_use_id="tool-123", name="calculator", arguments=query
    )

    # Process the result
    print(f"Calculation result: {result['content'][0]['text']}")

[10/29/25 06:24:01] INFO     Created new transport with session ID:                  ]8;id=642856;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=197399;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/server/streamable_http_manager.py#233\233]8;;\
                             84128748bc1b47ffb748d888fc7d1dbe                                                      

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=146018;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=210557;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Received session ID: 84128748bc1b47ffb748d888fc7d1dbe           ]8;id=398184;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=755418;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/client/streamable_http.py#134\134]8;;\

                    INFO     Negotiated protocol version: 2025-06-18                         ]8;id=575176;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=872575;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/client/streamable_http.py#146\146]8;;\

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 202 Accepted"   ]8;id=102546;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=389089;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/httpx/_client.py#1740\1740]8;;\

...

                    INFO     Processing request of type ListToolsRequest                              ]8;id=652837;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=913683;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/server/lowlevel/server.py#664\664]8;;\

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=295152;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=595743;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/httpx/_client.py#1740\1740]8;;\

Calculation result: 30


                    INFO     Terminating session: 84128748bc1b47ffb748d888fc7d1dbe           ]8;id=156406;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/server/streamable_http.py\streamable_http.py]8;;\:]8;id=674695;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/server/streamable_http.py#630\630]8;;\

                    INFO     HTTP Request: DELETE http://localhost:8000/mcp "HTTP/1.1 200 OK"       ]8;id=605887;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=699223;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/httpx/_client.py#1740\1740]8;;\

You can optionally also provide `read_timeout_seconds` while calling an MCP server tool to avoid it running for too long

In [9]:
with streamable_http_mcp_client:
    try:
        result = streamable_http_mcp_client.call_tool_sync(
            tool_use_id="tool-123",
            name="long_running_tool",
            arguments={"name": "Amazon"},
            read_timeout_seconds=timedelta(seconds=30),
        )

        if result["status"] == "error":
            print(f"Tool execution failed: {result['content'][0]['text']}")
        else:
            print(f"Tool execution succeeded: {result['content'][0]['text']}")
    except Exception as e:
        print(f"Tool call timed out or failed: {str(e)}")

[10/29/25 06:25:23] INFO     Created new transport with session ID:                  ]8;id=719610;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=507543;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/server/streamable_http_manager.py#233\233]8;;\
                             e03f826256484ad1aba3704fbf0bcee0                                                      

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=435781;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=3832;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Received session ID: e03f826256484ad1aba3704fbf0bcee0           ]8;id=666216;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=220910;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/client/streamable_http.py#134\134]8;;\

                    INFO     Negotiated protocol version: 2025-06-18                         ]8;id=366527;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=140162;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/client/streamable_http.py#146\146]8;;\

                    INFO     HTTP Request: GET http://localhost:8000/mcp "HTTP/1.1 200 OK"          ]8;id=535219;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=625655;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/httpx/_client.py#1740\1740]8;;\

...

[10/29/25 06:25:49] INFO     Processing request of type ListToolsRequest                              ]8;id=730695;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=936216;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/server/lowlevel/server.py#664\664]8;;\

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=799598;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=473499;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/httpx/_client.py#1740\1740]8;;\

Tool execution succeeded: Hello Amazon


                    INFO     Terminating session: e03f826256484ad1aba3704fbf0bcee0           ]8;id=55582;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/server/streamable_http.py\streamable_http.py]8;;\:]8;id=117845;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/mcp/server/streamable_http.py#630\630]8;;\

                    INFO     HTTP Request: DELETE http://localhost:8000/mcp "HTTP/1.1 200 OK"       ]8;id=477964;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=95726;file:///home/saeid/dev/spike/strands/strands-agents-samples/.venv/lib/python3.13/site-packages/httpx/_client.py#1740\1740]8;;\

### Interacting with multiple MCP servers

With Strands Agents you can also interact with multiple MCP servers using the same agent and configure tools setups such as the max number of tools that can be used in parallel (`max_parallel_tools`). Let's create a new agent to showcase this configuration:

<div style="text-align:center">
    <img src="./images/architecture_3.png" width="85%" />
</div>

In this agent, we will again use the AWS Documentation MCP server and we will also use the [AWS CDK MCP Server](https://awslabs.github.io/mcp/servers/cdk-mcp-server/) which helps with AWS Cloud Development Kit (CDK) best practices, infrastructure as code patterns and security compliance with CDK Nag.

First let's connect to the two MCP servers using the stdio transport

In [10]:
# Connect to an MCP server using stdio transport
aws_docs_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

# Connect to an MCP server using stdio transport
cdk_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(command="uvx", args=["awslabs.cdk-mcp-server@latest"])
    )
)

#### Create Agent with MCP servers

Next we will create the agent with the tools from both MCP servers

In [11]:
# Create an agent with MCP tools
with aws_docs_mcp_client, cdk_mcp_client:
    # Get the tools from the MCP server
    tools = aws_docs_mcp_client.list_tools_sync() + cdk_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(
        model=model,
        tools=tools)

    response = agent(
        "What is Amazon Bedrock pricing model. Be concise. Also what are the best practices related to CDK?"
    )

I'll help you with both questions. Let me search for Bedrock pricing information first and then provide CDK best practices.
Tool #1: search_documentation
Let me fetch the pricing details:
Tool #2: read_documentation
Now, let me get the CDK best practices:
Tool #3: CDKGeneralGuidance
Let me provide a concise summary of both topics:

**Amazon Bedrock Pricing Model:**
- Pay-as-you-go model based on actual usage
- Pricing is determined by:
  1. Volume of input tokens
  2. Volume of output tokens
  3. Whether you have Provisioned Throughput for the model
- Each model has its own pricing, which can be found in the Model providers page in the Bedrock console
- You only pay for the foundation models you actually use

**Key CDK Best Practices:**
1. Project Setup:
   - Always initialize projects properly using CDK CLI (`cdk init app`)
   - Use TypeScript or Python as preferred languages

2. Development Workflow:
   - Use `cdk synth` for validation
   - Use `cdk diff` to compare changes
   - Use 

### Congratulations!

In this notebook you learned how to connect with MCP servers using Strands Agent and two MCP transport protocols: stdio and Streamable HTTP. You also learned how to connect multiple MCP servers to the same agent. Next, let's see how to use different models with your agent